In [19]:
import tqdm, json, os, re, operator
import string, pprint, itertools, seaborn, collections, numpy as np, csv, requests, nltk
import JudgmentsProcessor
import JudgmentsClassifier

In [20]:
def process_judgment(judgment):
    data_to_query = (",").join(judgment)
    r = requests.post(data=data_to_query.encode("utf-8"), url="http://localhost:9200")
    response_text = r.text
    splited_response = response_text.splitlines()
    splited_response = [" ".join(x.replace("\t", " ").replace("none", "")[1:].split(":")[:2][:1]).replace(" ", ":")
                        for x in splited_response if ":" in x]
    splited_response = [x.split(":")[0] for x in splited_response]
    return splited_response

judgments = []
signatures = []
judgment_succeeded = []

def read_file(file_path, is_transforming=False):
   
    print ("Processing file" + file_path)
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in tqdm.tqdm(json_content):
            judgment = []
            if item['courtType'] in ['COMMON', 'SUPREME']:
                courtCases = item['courtCases']
                signatures_for_judgment = []
                for courtCase in courtCases:
                    signature = courtCase['caseNumber']
                    signatures_for_judgment.append(signature)
                signatures.append(signatures_for_judgment)
                item_count += 1
                text_content = re.sub("<.*?>", "", item["textContent"])
                text_content = text_content.replace('-\n', '')
                word_content = text_content.split()
                topicSpecificPunctuation = '„”–§…«»'
                translator = str.maketrans('', '', string.punctuation+topicSpecificPunctuation)

                for word in word_content:
                    word = word.translate(translator).lower()
                    if len(word)>0:
                        judgment.append(word)
                if not is_transforming:
                    judgments.append(judgment)
                else:
                    processed = process_judgment(judgment)
                    judgments.append(processed)
                    judgment_succeeded.append(len(judgment) > 0)

In [21]:
def read_all_judgments_from_2018(is_transforming=False):
        for filename in os.listdir("../data_filtered_2/"):
            read_file("../data_filtered_2/" + filename, is_transforming)

In [22]:
read_all_judgments_from_2018()

 47%|████▋     | 47/100 [00:00<00:00, 431.51it/s]

Processing file../data_filtered_2/judgments-3163.json
Processing file../data_filtered_2/judgments-3168.json


 27%|██▋       | 27/100 [00:00<00:00, 264.42it/s]

Processing file../data_filtered_2/judgments-3164.json


 39%|███▉      | 39/100 [00:00<00:00, 388.36it/s]

...

 36%|███▌      | 36/100 [00:00<00:00, 355.04it/s]

Processing file../data_filtered_2/judgments-3166.json


 30%|███       | 30/100 [00:00<00:00, 298.81it/s]

Processing file../data_filtered_2/judgments-3170.json


100%|██████████| 100/100 [00:00<00:00, 279.62it/s]


In [23]:
lists = [False] * 775
lists.append(True)
lists.append(True)
judge_processor = JudgmentsProcessor.JudgmentsProcessor(judgments, signatures, 'filteredRank.csv', lists)

In [24]:
X, Y, counter = judge_processor.process_judgments()

['na', 'do', 'nie', 'że', 'dnia', 'się', 'art', 'od', 'przez', 'sąd', 'roku', 'pracy', 'za', 'jest', 'oraz', 'zł', 'tym', 'to', 'sądu', 'co']


In [25]:
pprint.pprint (counter)

Counter({'sprawy z zakresu ubezpieczenia społecznego': 277,
         'sprawy cywilne': 244,
         'sprawy karne': 158,
         'sprawy gospodarcze': 120,
         'sprawy w zakresie prawa pracy': 76,
         'sprawy w zakresie prawa rodzinnego': 46,
         'sprawy o wykroczenia': 36,
         'sprawy w zakresie prawa konkurencji': 5})


In [26]:
print (len(X))
print (len(Y))

777
777


In [27]:
judgment_classifier = JudgmentsClassifier.JudgmentsClassifier(X, Y)
result_dict, clf, hyper_params = judgment_classifier.transform_and_train_classifier()

/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 8
  .format(len(labels), len(target_names))


In [28]:
print (hyper_params)
print(result_dict['accuracy_score'])
print(result_dict['classification_report'])
print(result_dict['micro_report'])
print(result_dict['macro_report'])

{'C': 15000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.902564102564
                                            precision    recall  f1-score   support

                            sprawy cywilne       0.83      0.89      0.86        54
sprawy z zakresu ubezpieczenia społecznego       1.00      1.00      1.00        57
                              sprawy karne       0.93      0.93      0.93        30
                        sprawy gospodarcze       0.67      0.80      0.73        15
             sprawy w zakresie prawa pracy       1.00      0.71      0.83        14
        sprawy w zakresie prawa rodzinnego       1.00      0.93      0.96        14
                      sprawy o wykroczenia       0.89      0.73      0.80        11

                               avg / total       0.91      0.90      0.90       195

(0.90256410256410258, 0.90256410256410258, 0.90256410256410269, None)
(0.90235358511220576, 0.85605029890744178, 0.87343503057788763, None)


In [29]:
import pickle
with open('judgments.pickle', 'rb') as handle:
    judgments = pickle.load(handle)
with open('signatures.pickle', 'rb') as handle:
    signatures = pickle.load(handle)
    
    judge_processor = JudgmentsProcessor.JudgmentsProcessor(judgments, signatures, 'taggedRank.csv', lists)

In [30]:
X, Y, counter = judge_processor.process_judgments()

['w', 'z', 'na', 'do', 'rok', 'być', 'dzień', 'on', 'sąd', 'praca', '1', 'ten', 'od', 'przez', 'który', 'o', 'art', 'k', 'sprawa', 'złoty']


In [31]:
judgment_classifier = JudgmentsClassifier.JudgmentsClassifier(X, Y)
result_dict, clf, hyper_params = judgment_classifier.transform_and_train_classifier()

/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 8
  .format(len(labels), len(target_names))


In [32]:
print (hyper_params)
print(result_dict['accuracy_score'])
print(result_dict['classification_report'])
print(result_dict['micro_report'])
print(result_dict['macro_report'])

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
0.913265306122
                                            precision    recall  f1-score   support

                            sprawy cywilne       0.81      0.90      0.85        49
sprawy z zakresu ubezpieczenia społecznego       1.00      1.00      1.00        48
                              sprawy karne       1.00      0.92      0.96        39
                        sprawy gospodarcze       0.83      0.83      0.83        23
             sprawy w zakresie prawa pracy       0.92      0.73      0.81        15
        sprawy w zakresie prawa rodzinnego       1.00      1.00      1.00        14
                      sprawy o wykroczenia       0.78      0.88      0.82         8

                               avg / total       0.92      0.91      0.91       196

(0.91326530612244894, 0.91326530612244894, 0.91326530612244894, None)
(0.90504945939728554, 0.89363662808649502, 0.89697144502001347, None)


In [33]:
# Remove from tagged dataset those where the tagger failed
# Test non-tagged and tagged on the very same dataset (including random_state!)